In [ ]:
from sklearn import datasets
from torch import nn
import numpy as np
import torch

In [ ]:
breast_cancer = datasets.load_breast_cancer()

breast_cancer_features_raw = breast_cancer.data
breast_cancer_labels_raw = breast_cancer.target

print(breast_cancer_features_raw.shape)

In [ ]:
def sigmoid(x):
    if -x > np.log(np.finfo(np.float32).max):
        return np.zeros_like(x)
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def ReLU(x):
    return np.maximum(0, x)

def ReLU_prime(x):
    return np.where(x > 0, 1, 0)

class BCELoss(object):
    @staticmethod
    def fn(output, target):
        return -np.mean(target * np.log(output) + (1 - target) * np.log(1 - output))
    
    @staticmethod
    def delta(output, target, activation_fn=sigmoid):
        if activation_fn == sigmoid:
            return output - target
        elif activation_fn == ReLU:
            return np.where(output > 0, output - target, 0)

class Hymmn0s_FNN():
    def __init__(self, sizes, activation_fn, loss):
        self.layers = len(sizes)
        self.sizes = sizes[1:]
        self.bias = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.activation_fn = activation_fn
        self.loss = loss

    def set_activation_fn_prime(self):
        self.activation_fn_prime = []
        for fn in self.activation_fn:
            if fn == sigmoid:
                self.activation_fn_prime.append(sigmoid_prime)
            elif fn == ReLU:
                self.activation_fn_prime.append(ReLU_prime)
            else:
                raise ValueError("Unsupported activation function")

    def forward(self, x):
        for i in range(self.layers - 1):
            x = self.activation_fn[i](np.dot(self.weights[i], x) + self.bias[i])
        return x
    
    '''
    @staticmethod
    def cost_derivative(output_activation, y):
        return output_activation - y
    '''
    
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.bias]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x.reshape(-1, 1) if len(x.shape) == 1 else x
        activations = [x]
        zs = []
        
        for i in range(self.layers - 1):
            z = np.dot(self.weights[i], activation) + self.bias[i]
            zs.append(z)
            activation = self.activation_fn[i](z)
            activation = activation.reshape(-1, 1)
            activations.append(activation)

        delta = self.loss.delta(activations[-1], y.reshape(-1, 1), self.activation_fn[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta.reshape(self.sizes[-1],1), activations[-2].reshape(1, self.sizes[-2]))

        for l in range(2, self.layers):
            z = zs[-l]
            sp = self.activation_fn_prime[-l](z)
            delta = np.dot(self.weights[-l + 1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta.reshape(self.sizes[-l],1), activations[-l - 1].reshape(1, len(activations[-l - 1])))

        return nabla_b, nabla_w
    
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.bias]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.bias = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.bias, nabla_b)]

    def SGD(self, training_data, epochs, mini_batch_size, eta):
        n = len(training_data)
        for i in range(epochs):
            np.random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            #print(f"Epoch {i + 1} complete")



In [ ]:
model = Hymmn0s_FNN([30, 8, 1], activation_fn=[ReLU,sigmoid], loss=BCELoss)

model.set_activation_fn_prime()
model.SGD(list(zip(breast_cancer_features_raw[:400], breast_cancer_labels_raw[:400])), epochs=200, mini_batch_size=20, eta=1e-5)

In [ ]:
def evaluate(model, test_data):
    test_results = [(1 if model.forward(x.reshape(-1,1)) > 0.5 else 0, y) for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

test_data = list(zip(breast_cancer_features_raw[400:], breast_cancer_labels_raw[400:]))
accuracy = evaluate(model, test_data)
print(f"Accuracy: {accuracy / len(test_data)}")

In [ ]:
class FNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(30, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        result = self.linear_relu_stack(x)
        return result
    
def train(model, data, labels):
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
    loss_fn = nn.BCELoss()

    for t in range(100000): 
        y_pred = model(data)
        #print(y_pred.view(-1))
        loss = loss_fn(y_pred, labels)

        #print(f"Epoch {t + 1}: {loss.item()}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def predict(model, data):
    model.eval()
    with torch.no_grad():
        y_pred = model(data)
        y_pred[y_pred < 0.5] = 0
        y_pred[y_pred >= 0.5] = 1
        return y_pred.view(-1)

def test(model, data, labels):
    model.eval()
    with torch.no_grad():
        y_pred = predict(model, data)
        correct = [1 for i in range(len(y_pred)) if y_pred[i] == labels[i]]
        accuracy = len(correct) / len(y_pred)
        print(f"Accuracy: {accuracy * 100}%")

In [ ]:
model = FNN().to('cuda' if torch.cuda.is_available() else 'cpu')

data = torch.tensor(breast_cancer_features_raw[:400], dtype=torch.float32)
labels = torch.tensor(breast_cancer_labels_raw[:400], dtype=torch.float32)
labels = labels.view(-1, 1)

train(model, data, labels)

In [ ]:
test_data = torch.tensor(breast_cancer_features_raw[400:], dtype=torch.float32)
test_labels = torch.tensor(breast_cancer_labels_raw[400:], dtype=torch.float32).view(-1, 1)

test(model, test_data, test_labels)